# Best Models 
Data used - full wins and full losses, skews fixed
- 30000 rows, first 11 columns, takes 9 minutes to run
    - Best: GradientBoostingClassifier(n_trees=100),Prec=0.556,Wins=503
- 400k rows, first 11 columns, took 2 hours to run
    - Best: GradientBoostingClassifier(n_trees=400),Prec=0.555,Wins=6733

In [1]:
import seaborn as sns, pandas as pd, numpy as np
import matplotlib.pyplot as plt, os, sys, seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import time
from datetime import datetime
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
import pprint
import pickle
from scipy import stats

In [2]:
def cross_val_summary(model, X_train, y_train):
    cross_val = cross_val_score(model, X_train, y_train, cv=5,scoring='accuracy')
    print('cross Validation score:',round(cross_val.mean(),3),[round(num, 3) for num in cross_val])
    return None

def performance_scores(model,y_test, y_pred):
    cross_val_summary(model, X_train, y_train)
    prec_score = round(precision_score(y_test, y_pred), 3)
    acc_score = round(accuracy_score(y_test, y_pred), 3)
#     matrix = confusion_matrix(y_test, y_pred)
#     sns.set_palette(sns.color_palette())
#     _, ax = plt.subplots(figsize=(3,3))
#     ax = sns.heatmap(matrix, annot=True, fmt='d', annot_kws={"size": 20, "weight": "bold"})  
#     labels = ['Over', 'Under']
#     ax.set_xticklabels(labels, fontsize=10);
#     ax.set_yticklabels(labels, fontsize=10);
#     ax.set_ylabel('Target', fontsize=10);
#     ax.set_xlabel('Prediction', fontsize=10)
    print('Prec:',prec_score,'Acc:',acc_score)
    model_results.append(pd.Series({
                               'Model': type(model).__name__, 
                               'Model_Details': model, 
                               'Prec': prec_score,
                               'Acc': acc_score}))
    return None

In [3]:
start = datetime.now()
model_results = list()

In [4]:
df_all_data = pd.read_csv('data1').iloc[:,1:]
print(df_all_data.shape)
df_all_data.head(1)

(75841, 16)


,match_id,match_start,league,home,away,final_goals,minute,line,price,current_goals,minute_bin,profit,shots,eff_line,da_perc,target
0,4243211,2021-10-28 17:45:00,denmark cup,midtjylland,aab,4.0,6,2.5,1.925,0.0,10,-1.0,0.0,2.5,0.73,0


In [5]:
df_all_data['minute_bin'] = df_all_data['minute_bin'].apply(lambda x: str(x))

In [6]:
df = df_all_data[['current_goals',
       'shots', 'da_perc', 'eff_line', 'minute_bin','target']]
df.head(1)

,current_goals,shots,da_perc,eff_line,minute_bin,target
0,0.0,0.0,0.73,2.5,10,0


In [7]:
df = pd.get_dummies(df)

In [8]:
df.head()

,current_goals,shots,da_perc,eff_line,target,minute_bin_10,minute_bin_20,minute_bin_30,minute_bin_40,minute_bin_50,minute_bin_60,minute_bin_70,minute_bin_80
0,0.0,0.0,0.73,2.5,0,1,0,0,0,0,0,0,0
1,0.0,1.0,0.60,2.5,1,1,0,0,0,0,0,0,0
2,0.0,0.0,0.62,2.5,1,1,0,0,0,0,0,0,0
3,0.0,0.0,0.80,2.5,1,1,0,0,0,0,0,0,0
4,0.0,1.0,0.75,2.5,1,1,0,0,0,0,0,0,0


In [9]:
mask = df.dtypes == np.float
float_cols = df.columns[mask]

skew_limit = 0.75 # define a limit above which we will transform
skew_vals = df[float_cols].skew()

In [10]:
skew_cols = (skew_vals
             .sort_values(ascending=False)
             .to_frame()
             .rename(columns={0:'Skew'})
             .query('abs(Skew) > {}'.format(skew_limit)))
log,sq_rt,box_cox = [],[],[]
for x in skew_cols.index:
    log.append(np.log1p(df[x]).skew())
    sq_rt.append(np.sqrt(df[x]).skew())

skew_cols['log'] = log
skew_cols['sq_rt'] = sq_rt

skew_cols

,Skew,log,sq_rt
da_perc,1.130335,0.960638,0.925872


In [11]:
for col in skew_cols.index.values:
    df[col] = df[col].apply(np.sqrt)

In [12]:
list(skew_cols.index)

['da_perc']

In [13]:
from sklearn.model_selection import StratifiedShuffleSplit

target_variable = 'target'
feature_cols = df.drop([target_variable],1).columns

# Get the split indexes
strat_shuf_split = StratifiedShuffleSplit(n_splits=1, 
                                          test_size=0.3, 
                                          random_state=42)

train_idx, test_idx = next(strat_shuf_split.split
                           (df[feature_cols], df[target_variable]))

# Create the dataframes
X_train = df.loc[train_idx, feature_cols]
y_train = df.loc[train_idx, target_variable]

X_test  = df.loc[test_idx, feature_cols]
y_test  = df.loc[test_idx, target_variable]

In [14]:
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)
print(round(y_train.mean(),3),round(y_test.mean(),3))

(53088, 12) (53088,) (22753, 12) (22753,)
0.489 0.489


In [15]:
y_train.value_counts(True),y_test.value_counts(True)

(0    0.510737
 1    0.489263
 Name: target, dtype: float64,
 0    0.510746
 1    0.489254
 Name: target, dtype: float64)

In [16]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# scaler = MinMaxScaler()
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Logistic Regression

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score,precision_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV



In [18]:
parameters = {
    'penalty' : ['l1','l2'], 
    'C'       : [1,1e-01,1e-02,1e-03],
    'solver'  : ['newton-cg', 'lbfgs', 'liblinear','saga'],
}

logreg = LogisticRegression(random_state=42)
lr = GridSearchCV(logreg,                    # model
                   param_grid = parameters,   # hyperparameters
                   scoring='accuracy',        # metric for scoring
                   cv=5
                   )                   
lr.fit(X_train,y_train)

print(lr.best_score_, lr.best_params_)
best_lr = lr.best_estimator_.fit(X_train,y_train)
y_pred = best_lr.predict(X_test)

performance_scores(best_lr,y_test, y_pred)

coef_df = pd.DataFrame(df.drop('target',1).columns,columns=['feature'])
coef_df['importance'] = abs(best_lr.coef_).squeeze().tolist()
coef_df.sort_values('importance',ascending=False)

0.5234137835678051 {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}
cross Validation score: 0.523 [0.53, 0.523, 0.526, 0.519, 0.519]
Prec: 0.51 Acc: 0.517


,feature,importance
0,current_goals,0.110476
4,minute_bin_10,0.099635
3,eff_line,0.062659
10,minute_bin_70,0.058234
11,minute_bin_80,0.054113
5,minute_bin_20,0.046918
8,minute_bin_50,0.045808
1,shots,0.042743
9,minute_bin_60,0.041769
2,da_perc,0.007583


# KNN

In [19]:
from sklearn.neighbors import KNeighborsClassifier
for weights in ['uniform','distance']:
    for k in [1,25,50,100]:

        knn = KNeighborsClassifier(n_neighbors=k, weights=weights)
        knn = knn.fit(X_train, y_train)

        y_pred = knn.predict(X_test)
        prec_score = round(precision_score(y_test, y_pred), 2)
        print('K_neighbours:',k,'Weight:',weights)
        performance_scores(knn,y_test, y_pred)


K_neighbours: 1 Weight: uniform
cross Validation score: 0.502 [0.504, 0.504, 0.508, 0.499, 0.496]
Prec: 0.491 Acc: 0.501
K_neighbours: 25 Weight: uniform
cross Validation score: 0.507 [0.508, 0.503, 0.51, 0.504, 0.509]
Prec: 0.492 Acc: 0.503
K_neighbours: 50 Weight: uniform
cross Validation score: 0.515 [0.522, 0.514, 0.52, 0.508, 0.509]
Prec: 0.497 Acc: 0.508
K_neighbours: 100 Weight: uniform
cross Validation score: 0.515 [0.521, 0.516, 0.518, 0.508, 0.511]
Prec: 0.498 Acc: 0.51
K_neighbours: 1 Weight: distance
cross Validation score: 0.502 [0.504, 0.504, 0.508, 0.499, 0.496]
Prec: 0.491 Acc: 0.501
K_neighbours: 25 Weight: distance
cross Validation score: 0.507 [0.513, 0.501, 0.517, 0.501, 0.501]
Prec: 0.485 Acc: 0.497
K_neighbours: 50 Weight: distance
cross Validation score: 0.511 [0.517, 0.506, 0.525, 0.505, 0.503]
Prec: 0.492 Acc: 0.504
K_neighbours: 100 Weight: distance
cross Validation score: 0.511 [0.519, 0.507, 0.524, 0.504, 0.501]
Prec: 0.489 Acc: 0.501


# SGDClassifier

In [20]:
from sklearn.linear_model import SGDClassifier

alphas = [1e-01,1e-02]
for loss_type in ['log','modified_huber']:
    for alpha in alphas:
        sgd = SGDClassifier(loss=loss_type,alpha=alpha,random_state=42)
        sgd = sgd.fit(X_train, y_train)

        y_pred = sgd.predict(X_test)
        prec_score = round(precision_score(y_test, y_pred), 2)
        print('Loss Type:',loss_type,'Alpha',alpha)

        performance_scores(sgd,y_test, y_pred)


Loss Type: log Alpha 0.1
cross Validation score: 0.521 [0.53, 0.521, 0.524, 0.518, 0.514]
Prec: 0.509 Acc: 0.516
Loss Type: log Alpha 0.01
cross Validation score: 0.523 [0.526, 0.525, 0.525, 0.52, 0.517]
Prec: 0.502 Acc: 0.513
Loss Type: modified_huber Alpha 0.1
cross Validation score: 0.523 [0.525, 0.525, 0.526, 0.52, 0.52]
Prec: 0.503 Acc: 0.514
Loss Type: modified_huber Alpha 0.01
cross Validation score: 0.52 [0.519, 0.524, 0.519, 0.528, 0.513]
Prec: 0.503 Acc: 0.513


# Decision Tree Classifier

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)



param_grid = {'criterion' : ["gini", "entropy"],
              'max_depth': [1,2,3,4,5],
              'max_features': [2,5,10,20,30]}

best_dt = GridSearchCV(dt,
                  param_grid=param_grid,
                  scoring='precision',
#                   n_jobs=-1,
                  cv=5)

best_dt = best_dt.fit(X_train, y_train)


In [22]:
print(best_dt.best_score_, best_dt.best_params_)

best_dt1 = best_dt.best_estimator_.fit(X_train,y_train)
y_pred = best_dt1.predict(X_test)
performance_scores(best_dt1,y_test, y_pred)


0.5223137216565952 {'criterion': 'gini', 'max_depth': 3, 'max_features': 5}
cross Validation score: 0.522 [0.53, 0.524, 0.524, 0.519, 0.515]
Prec: 0.506 Acc: 0.514


# Random Forest Classifier

In [23]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(oob_score=True, 
                            random_state=42, 
                            warm_start=True,
                            n_jobs=-1)

for n_trees in [200,300,400]:
    
    RF.set_params(n_estimators=n_trees)

    RF.fit(X_train, y_train)
    
    y_pred = RF.predict(X_test)
    prec_score = round(precision_score(y_test, y_pred), 2)
    print('N_estimators:',n_trees)
    performance_scores(RF,y_test, y_pred)


N_estimators: 200
cross Validation score: 0.509 [0.516, 0.509, 0.52, 0.5, 0.502]
Prec: 0.49 Acc: 0.502
N_estimators: 300
cross Validation score: 0.509 [0.515, 0.508, 0.52, 0.5, 0.502]
Prec: 0.492 Acc: 0.504
N_estimators: 400
cross Validation score: 0.509 [0.515, 0.507, 0.521, 0.501, 0.502]
Prec: 0.491 Acc: 0.503


# Extra Trees Classifier

In [24]:
from sklearn.ensemble import ExtraTreesClassifier
ET = ExtraTreesClassifier(
#                             oob_score=True, 
                            random_state=42, 
                            warm_start=True,
#                             n_jobs=-1
                           )

for n_trees in [5,50,100]:
    
    ET.set_params(n_estimators=n_trees)

    ET.fit(X_train, y_train)
    
    y_pred = ET.predict(X_test)
    prec_score = round(precision_score(y_test, y_pred), 2)
    print('N_estimators:',n_trees)

    performance_scores(ET,y_test, y_pred)


N_estimators: 5
cross Validation score: 0.51 [0.517, 0.508, 0.522, 0.504, 0.502]
Prec: 0.491 Acc: 0.503
N_estimators: 50
cross Validation score: 0.511 [0.516, 0.507, 0.524, 0.504, 0.502]
Prec: 0.491 Acc: 0.503
N_estimators: 100
cross Validation score: 0.511 [0.516, 0.508, 0.524, 0.504, 0.502]
Prec: 0.491 Acc: 0.503


# Gradient Boosting Classifier

In [25]:
from sklearn.ensemble import GradientBoostingClassifier

tree_list = [5,50,100]
for n_trees in tree_list:
    
    GBC = GradientBoostingClassifier(max_features = 10, 
                    n_estimators=n_trees, random_state=42)

    GBC.fit(X_train, y_train)
    y_pred = GBC.predict(X_test)
    print('N_estimators:',n_trees)

    performance_scores(GBC,y_test, y_pred)


N_estimators: 5
cross Validation score: 0.518 [0.52, 0.518, 0.52, 0.514, 0.517]
Prec: 0.504 Acc: 0.513
N_estimators: 50
cross Validation score: 0.521 [0.526, 0.519, 0.527, 0.518, 0.515]
Prec: 0.507 Acc: 0.516
N_estimators: 100
cross Validation score: 0.521 [0.526, 0.522, 0.523, 0.519, 0.515]
Prec: 0.513 Acc: 0.52


In [26]:
from sklearn.model_selection import GridSearchCV

# The parameters to be fit
param_grid = {
    'learning_rate': [20, 10, 5, 3, 1, 0.1, ],
              'subsample': [2, 1.0, 0.5, 0.25],
              'max_features': [3, 4, 5, 10]
}

# The grid search object
GV_GBC = GridSearchCV(GradientBoostingClassifier(n_estimators = 100,random_state=42), 
                      param_grid=param_grid, 
                      scoring='precision',
                      n_jobs=-1)

# Do the grid search
GV_GBC = GV_GBC.fit(X_train, y_train)

print(GV_GBC.best_score_, GV_GBC.best_params_)

GV_GBC = GV_GBC.best_estimator_.fit(X_train,y_train)
y_pred = GV_GBC.predict(X_test)

performance_scores(GV_GBC,y_test, y_pred)


0.5181099534596944 {'learning_rate': 10, 'max_features': 5, 'subsample': 1.0}
cross Validation score: 0.507 [0.494, 0.513, 0.514, 0.511, 0.504]
Prec: 0.496 Acc: 0.506


# AdaBoost Classifier

In [27]:
from sklearn.ensemble import AdaBoostClassifier

ABC = AdaBoostClassifier()

param_grid = {'n_estimators': [10,30,50,100],
              'learning_rate': [5,1,0.1, 0.01, 0.001]}

GV_ABC = GridSearchCV(ABC,
                      param_grid=param_grid, 
                      scoring='precision',
                      n_jobs=-1)

GV_ABC = GV_ABC.fit(X_train, y_train)

print(GV_ABC.best_score_, GV_ABC.best_params_)

GV_ABC = GV_ABC.best_estimator_.fit(X_train,y_train)
y_pred = GV_ABC.predict(X_test)

performance_scores(GV_ABC,y_test, y_pred)

0.5155986894703991 {'learning_rate': 0.1, 'n_estimators': 50}
cross Validation score: 0.523 [0.532, 0.523, 0.527, 0.519, 0.514]
Prec: 0.503 Acc: 0.514


In [28]:
df = (pd.DataFrame(model_results).sort_values('Acc',ascending=False)
      .reset_index().iloc[:,1:].head(30))


In [29]:
chosen_models = df.drop_duplicates(['Model']).reset_index().iloc[:,1:].head(6)
chosen_models

,Model,Model_Details,Prec,Acc
0,GradientBoostingClassifier,([DecisionTreeRegressor(criterion='friedman_ms...,0.513,0.520
1,LogisticRegression,"LogisticRegression(C=0.1, penalty='l1', random...",0.510,0.517
2,SGDClassifier,"SGDClassifier(alpha=0.1, loss='log', random_st...",0.509,0.516
3,DecisionTreeClassifier,"DecisionTreeClassifier(max_depth=3, max_featur...",0.506,0.514
4,AdaBoostClassifier,"(DecisionTreeClassifier(max_depth=1, random_st...",0.503,0.514
5,KNeighborsClassifier,KNeighborsClassifier(n_neighbors=100),0.498,0.510


In [30]:
chosen_models = chosen_models['Model_Details']

# Voting Classifier

In [31]:
import itertools
from sklearn.ensemble import VotingClassifier

max_est = range(5)
for ind_est in range(3, len(max_est)+1):
    for subset in itertools.combinations(max_est, ind_est):
        estimators = []
        for est in subset:
            estimators.append(['est'+str(est),chosen_models[est]])
        print('\n',str(np.array(estimators)[:,0]))
        VC = VotingClassifier(estimators, voting='soft')
        VC = VC.fit(X_train, y_train)
        y_pred = VC.predict(X_test)
        performance_scores(VC,y_test, y_pred)



 ['est0' 'est1' 'est2']
cross Validation score: 0.523 [0.528, 0.525, 0.523, 0.52, 0.517]
Prec: 0.51 Acc: 0.518

 ['est0' 'est1' 'est3']
cross Validation score: 0.524 [0.53, 0.525, 0.527, 0.522, 0.517]
Prec: 0.509 Acc: 0.516

 ['est0' 'est1' 'est4']
cross Validation score: 0.522 [0.53, 0.524, 0.523, 0.519, 0.515]
Prec: 0.512 Acc: 0.52

 ['est0' 'est2' 'est3']
cross Validation score: 0.525 [0.531, 0.526, 0.529, 0.524, 0.517]
Prec: 0.505 Acc: 0.514

 ['est0' 'est2' 'est4']
cross Validation score: 0.522 [0.527, 0.525, 0.525, 0.518, 0.516]
Prec: 0.509 Acc: 0.517

 ['est0' 'est3' 'est4']
cross Validation score: 0.525 [0.532, 0.526, 0.527, 0.523, 0.515]
Prec: 0.507 Acc: 0.515

 ['est1' 'est2' 'est3']
cross Validation score: 0.522 [0.53, 0.523, 0.523, 0.518, 0.514]
Prec: 0.505 Acc: 0.513

 ['est1' 'est2' 'est4']
cross Validation score: 0.523 [0.531, 0.523, 0.526, 0.519, 0.517]
Prec: 0.509 Acc: 0.516

 ['est1' 'est3' 'est4']
cross Validation score: 0.522 [0.53, 0.523, 0.523, 0.518, 0.514]
Prec

In [32]:
vc_models = (pd.DataFrame(model_results).sort_values('Acc',ascending=False)
      .reset_index().iloc[:,1:])


In [33]:
vc_models = (vc_models.loc[vc_models['Model']=='VotingClassifier']
.reset_index().iloc[:,1:])
vc_models.head(5)

,Model,Model_Details,Prec,Acc
0,VotingClassifier,"VotingClassifier(estimators=[['est0',\n ...",0.512,0.520
1,VotingClassifier,"VotingClassifier(estimators=[['est0',\n ...",0.510,0.518
2,VotingClassifier,"VotingClassifier(estimators=[['est0',\n ...",0.509,0.517
3,VotingClassifier,"VotingClassifier(estimators=[['est0',\n ...",0.509,0.517
4,VotingClassifier,"VotingClassifier(estimators=[['est0',\n ...",0.508,0.516
